In [ ]:
#53424 users 53425 book is 2757 or 2756
#22 final genres

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import time

In [63]:
books = pd.read_csv('books.csv')

In [68]:
# books.book_id = pd.to_numeric(books.book_id)

In [102]:
# books[books.book_id.isin([144,  150, 1361,  100,   19,  669,  109,  143])]['title']

18      The Fellowship of the Ring (The Lord of the Ri...
99                                   The Poisonwood Bible
108                                        Les Misérables
142                           All the Light We Cannot See
143     Unbroken: A World War II Story of Survival, Re...
149                                          The Red Tent
668                              The House of the Spirits
1360                                The House at Riverton
Name: title, dtype: object

In [97]:
# books.columns

Index([u'book_id', u'goodreads_book_id', u'best_book_id', u'work_id',
       u'books_count', u'isbn', u'isbn13', u'authors',
       u'original_publication_year', u'original_title', u'title',
       u'language_code', u'average_rating', u'ratings_count',
       u'work_ratings_count', u'work_text_reviews_count', u'ratings_1',
       u'ratings_2', u'ratings_3', u'ratings_4', u'ratings_5', u'image_url',
       u'small_image_url'],
      dtype='object')

In [2]:
#Parses the LibFM output model, returns w0, wj, and vj,f.
#w0 is a single number, wj is an array of floats, vj,f is an array of array of floats
def parse_output_file(filename):
    w0_spot=False
    wj_spot=False
    vj_spot=False
    w0=0
    wj=[]
    vj=[]
    f = open(filename, 'r')
    for line in f:
        if 'global bias' in line:
            w0_spot=True
        elif w0_spot:
            w0=float(line)
            w0_spot=False
        elif 'unary interactions' in line:
            wj_spot=True
        elif 'pairwise interactions' in line:
            wj_spot=False
            vj_spot=True
        elif wj_spot:
            wj.append(float(line))
        elif vj_spot:
            line_split=line.split(' ')
            line_numbers = [float(i) for i in line_split]
            vj.append(line_numbers)
    return w0,wj,vj

In [3]:
w0, wj, vj = parse_output_file('../libfm_writes/model1.libfm')

In [4]:
num_col=len(wj)

In [37]:
#Given a string (line from the libfm testing set), outputs an input vector
def create_input_vector(sample, num_col): 
    input_vector=[0]*num_col
    sample_split = sample.split(' ')
    sample_split = sample_split[1:]
    for s in sample_split:
        col_val = int(s[:s.index(':')])
        item_val = float(s[s.index(':')+1:])
        input_vector[col_val]=item_val
    return input_vector


In [38]:
sample = '3 42562:1 56182:1 63432:0.01118210862619808 63434:0.02396166134185303 63439:0.2220447284345048 63442:0.7092651757188498 63443:0.007987220447284345 63456:0.0255591054313099'
input_vector = create_input_vector(sample, num_col)
#4.61783

In [39]:
input_vector = np.array(input_vector)
wj = np.array(wj)
vj = np.array(vj)

### Testing with 10k books and given user

In [5]:
books_genres = sparse.load_npz('books_genres.npz')

In [11]:
# sparse.find(books_genres[2756])

(array([0, 0, 0, 0, 0, 0, 0], dtype=int32),
 array([ 2757, 10007, 10009, 10014, 10017, 10018, 10031], dtype=int32),
 array([ 1.        ,  0.01118211,  0.02396166,  0.22204473,  0.70926518,
         0.00798722,  0.02555911]))

In [16]:
books_genres.shape

(10000, 10041)

### Setting test user to be 42562
I'll put this in the function where we can call the user id

In [14]:
users = sparse.csr_matrix((10000, 53425), dtype=int)

In [17]:
users[:,42562] = 1

/Users/adamcoviensky/anaconda/lib/python2.7/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [23]:
def concatenate_csc_matrices_by_columns(matrix1, matrix2):
    new_data = np.concatenate((matrix1.data, matrix2.data))
    new_indices = np.concatenate((matrix1.indices, matrix2.indices))
    new_ind_ptr = matrix2.indptr + len(matrix1.data)
    new_ind_ptr = new_ind_ptr[1:]
    new_ind_ptr = np.concatenate((matrix1.indptr, new_ind_ptr))

    return sparse.csc_matrix((new_data, new_indices, new_ind_ptr))

In [25]:
X = concatenate_csc_matrices_by_columns(users.tocsc(), books_genres.tocsc())

In [28]:
X = X.tocsr()

In [32]:
X[0,:]

<1x63466 sparse matrix of type '<type 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

### Own Sparse Creation

In [10]:
def make_prediction(x, V, w, w0):
    """Function to create predictions
    Input:
    x: feature vector
    V: interaction term matrix
    w: linear feature biases (array)
    w0: global bias (float)
    Returns:
    predicted rating (float)"""
    x = sparse.csr_matrix(x)
    x2 = (x.T).dot(x)
    x_upper = sparse.triu(x2, k=1)
    pred_inter = 0
    my_tup_upper = sparse.find(x_upper)
    for entry in range(my_tup_upper[0].shape[0]):
    #    print(entry)
        i = my_tup_upper[0][entry]
        j = my_tup_upper[1][entry]
        value = my_tup_upper[2][entry]
        pred_inter += value*((vj[i,:]).dot(vj[j,:]))
    pred_linear = x.dot(w)
    return w0 + pred_linear + pred_inter

In [35]:
def make_prediction_2(x, V, w, w0):
    """Function to create predictions
    Input:
    x: feature vector
    V: interaction term matrix
    w: linear feature biases (array)
    w0: global bias (float)
    Returns:
    predicted rating (float)"""
    start_time = time.time()
    for i in range(10000):
#        print(i)
        x = sparse.csr_matrix(x)
        x2 = (x.T).dot(x)
        x_upper = sparse.triu(x2, k=1)
        pred_inter = 0
        my_tup_upper = sparse.find(x_upper)
        for entry in range(my_tup_upper[0].shape[0]):
        #    print(entry)
            i = my_tup_upper[0][entry]
            j = my_tup_upper[1][entry]
            value = my_tup_upper[2][entry]
            pred_inter += value*((vj[i,:]).dot(vj[j,:]))
        pred_linear = x.dot(w)
    print(time.time() - start_time)
    return w0 + pred_linear + pred_inter

In [99]:
def output_top_k(x_mat, V, w, w0, k):
    """Function to create predictions
    Input:
    x: matrix with 10k books for one specific user (around 63k by 10k)
    V: interaction term matrix
    w: linear feature biases (array)
    w0: global bias (float)
    Returns:
    predicted rating (float)"""
    start_time = time.time()
    pred_mat = np.zeros(10000)
    for i in range(10000):
        x = x_mat[i,:]
        x2 = (x.T).dot(x)
        x_upper = sparse.triu(x2, k=1)
        pred_inter = 0
        my_tup_upper = sparse.find(x_upper)
        for entry in range(my_tup_upper[0].shape[0]):
            row = my_tup_upper[0][entry]
            col = my_tup_upper[1][entry]
            value = my_tup_upper[2][entry]
            pred_inter += value*((vj[row,:]).dot(vj[col,:]))
        pred_linear = x.dot(w)
        pred_mat[i] = w0 + pred_linear + pred_inter
    
    # returning top k
    top_ids = np.argsort(-pred_mat)[:k]
    top_books = []
    for i in top_ids:
        top_books.append(books.loc())
    top_books = books[books.book_id.isin(top_ids)]['title']
    print(time.time() - start_time)
    return top_books

In [100]:
my_preds = output_top_k(X,vj,wj,w0, 5)

13.6168551445


In [101]:
my_preds

1008                                     The Interestings
6588                              Iceberg (Dirk Pitt, #3)
6918                                 Napalm & Silly Putty
8976    Drop Dead Healthy: One Man's Humble Quest for ...
9564                            Winter Stroll (Winter #2)
Name: title, dtype: object

### Testing shit

In [37]:
vj[2].dot(vj[4])

0.61170599452520003

In [38]:
wj.dot(input_vector)

0.034829011896823088

In [39]:
input_vector.shape

(63466,)

In [41]:
x2 = (x.T).dot(x)

In [42]:
x2.shape

(63466, 63466)

In [43]:
my_tup = sparse.find(x2)

In [44]:
my_tup

(array([10714, 60589, 63429, 63437, 63438, 63439, 63442, 63444, 63449,
        63451, 63456, 63458, 63465, 10714, 60589, 63429, 63437, 63438,
        63439, 63442, 63444, 63449, 63451, 63456, 63458, 63465, 10714,
        60589, 63429, 63437, 63438, 63439, 63442, 63444, 63449, 63451,
        63456, 63458, 63465, 10714, 60589, 63429, 63437, 63438, 63439,
        63442, 63444, 63449, 63451, 63456, 63458, 63465, 10714, 60589,
        63429, 63437, 63438, 63439, 63442, 63444, 63449, 63451, 63456,
        63458, 63465, 10714, 60589, 63429, 63437, 63438, 63439, 63442,
        63444, 63449, 63451, 63456, 63458, 63465, 10714, 60589, 63429,
        63437, 63438, 63439, 63442, 63444, 63449, 63451, 63456, 63458,
        63465, 10714, 60589, 63429, 63437, 63438, 63439, 63442, 63444,
        63449, 63451, 63456, 63458, 63465, 10714, 60589, 63429, 63437,
        63438, 63439, 63442, 63444, 63449, 63451, 63456, 63458, 63465,
        10714, 60589, 63429, 63437, 63438, 63439, 63442, 63444, 63449,
      

In [45]:
x_upper = sparse.triu(x2, k=1)

In [46]:
my_tup_upper = sparse.find(x_upper)

In [47]:
my_tup_upper[0].shape[0]

78

In [48]:
print(my_tup_upper[0][3])
print(my_tup_upper[1][3])
print(my_tup_upper[2][3])

10714
63437
0.0139900903527


In [49]:
vj[10714].dot(vj[63437])*0.01399

0.00010348149048362307

In [50]:
my_tup[2].shape[0]

169

In [51]:
my_tup[0][0]

10714

In [52]:
vj[i].shape

(8,)

### Attempting with trick from paper

In [75]:
vx = input_vector.dot(vj[:,0])

In [77]:
vx_sq = np.square(input_vector).dot(np.square(vj[:,0]))

In [82]:
vx_sq**2

0.24649971740344628

In [80]:
k = vj.shape[1]

In [13]:
def predict_fm(x, w0, w, V):
    start_time = time.time()
    for i in range(10000):
        my_sum = 0
        for f in range(vj.shape[1]):
            vx = (x.dot(vj[:,f]))**2
            vx_sq = np.square(x).dot(np.square(vj[:,f]))
            my_sum += (vx - vx_sq)
        interaction_term = my_sum/2.0
        linear_term = w.dot(x)
    print(time.time() - start_time)
    return w0 + linear_term + interaction_term

In [33]:
def predict_fm_2(x, w0, w, V):
    start_time = time.time()
    for i in range(10000):
        print(i)
        vx = (vj.T).dot(x)
        vx_sq = np.square(vj.T).dot(x.power(2))
        interaction_term = (vx - vx_sq).sum()/2
        linear_term = w.dot(x)
    print(time.time() - start_time)
    return w0 + linear_term + interaction_term

In [34]:
predict_fm_2(sparse.csr_matrix(input_vector), w0, wj, vj)

0


KeyboardInterrupt: 

In [30]:
sparse_x = sparse.csr_matrix(input_vector)
sparse_x.shape

(1, 63466)

In [15]:
vj.shape

(63466, 15)

In [29]:
vj.shape[1]

15

In [88]:
linear_term = wj[:,0].dot(input_vector)

In [89]:
linear_term

0.034829011896823088

In [90]:
w0 + linear_term + interaction_term

3.5977711975764448

In [91]:
input_vector

array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.01136695])